In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [ ]:
demo = pd.read_csv("PTDEMOG_12Apr2024.csv")

In [ ]:
neuro = pd.read_csv("NEUROEXM_12Apr2024.csv")

In [ ]:
neuro.columns

In [ ]:
clinical = pd.read_csv("adsp.csv")#.rename(columns={"PHASE":"Phase"})

In [ ]:
clinical.head()

In [ ]:
diag["Subject"].value_counts()

In [ ]:
comb = pd.read_csv("CLINICAL_DATA.csv")[["RID", "PTID" , "PHASE"]]

In [ ]:
m = comb.merge(demo, on = ["RID", "PHASE"]).merge(neuro,on = ["RID", "PHASE"]).merge(clinical,on = ["RID", "PHASE"]).drop_duplicates()

In [ ]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [ ]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [ ]:
m = m.merge(diag, on = ["PTID", "PHASE"])

In [ ]:
m["PTID"].value_counts()

In [ ]:
t = m

In [ ]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE","DX"], axis=1)

In [ ]:
t.columns

In [ ]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [ ]:
len(t.columns)
t.columns

In [ ]:
categorical = ['PTGENDER',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [ ]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [ ]:
text = ["CMMED"]

In [ ]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "PHASE", "RID", "PTID"]))
t[cols_left]

In [ ]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

In [ ]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY"]
t = t.drop(to_del, axis=1)

In [ ]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "PHASE", "RID", "PTID"]))
t[quant]

In [ ]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [ ]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'PHASE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT',
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [ ]:
t[l]

In [ ]:
dfs = []

In [ ]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [ ]:
cat = pd.concat(dfs, axis=1)

In [ ]:
t[quant]

In [ ]:
cat

In [ ]:
t[["PTID","RID", "PHASE", "Group"]]

In [ ]:
c = pd.concat([t[["PTID", "RID", "PHASE", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [ ]:
c

In [ ]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID',
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [ ]:
c.to_csv("clinical.csv")

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "PHASE", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
pd.DataFrame(X_train).to_pickle("X_train_c.pkl")
pd.DataFrame(y_train).to_pickle("y_train_c.pkl")

pd.DataFrame(X_test).to_pickle("X_test_c.pkl")
pd.DataFrame(y_test).to_pickle("y_test_c.pkl")